In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

df = pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")
df.head()

In [ ]:
print(df['Seller_Type'].unique())
print(df['Owner'].unique())
print(df['Transmission'].unique())

In [ ]:
df['Current_Year']=2021

In [ ]:
df['year']= df['Current_Year']-df['Year']

In [ ]:
df.drop(["Year"], axis=1 , inplace=True)

In [ ]:
df.drop(["Current_Year"], axis=1 , inplace=True)

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df

In [ ]:
final_dataset=df[[ 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner',
       'year']]

In [ ]:
final_dataset =pd.get_dummies(final_dataset, drop_first=True)
final_dataset.head()

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
corrmat = final_dataset.corr()
sns.heatmap(corrmat, annot=True)

In [ ]:
x=final_dataset.iloc[:,1:]
y=final_dataset.iloc[:,0]

In [ ]:
! pip install sklearn

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
feat_imp =pd.Series(model.feature_importances_, index=x.columns)
feat_imp


In [ ]:
feat_imp.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random=RandomForestRegressor()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start=100, stop=1200, num=12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#Randomized Search CV


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)


In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(x_test)


In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)